<a href="https://colab.research.google.com/github/talgalper/Honours-2021/blob/main/nextstrain_data_treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install ncbi-blast+
!apt-get install exonerate
!apt-get install dssp
!pip install biostructmap

In [39]:
import pandas as pd
from collections import defaultdict
import biostructmap

In [111]:
def nextstrain_data_processing(filename, seq_len, gene_id):
    ns_data = pd.read_csv(filename, sep='\t')

    s_gene_data = ns_data[ns_data.gene == gene_id]

    pos = s_gene_data.position

    entropy = s_gene_data.entropy

    # Default dict will return zero if not in original entropy data.
    pos_to_entropy = defaultdict(float, zip(pos, entropy))

    len_of_ref_seq = seq_len

    # Create a list that includes zeros.
    list_of_values = [pos_to_entropy[i+1] for i in range(len_of_ref_seq)]
    return list_of_values

In [112]:
s_protein_data = nextstrain_data_processing('nextstrain_ncov_global_diversity.tsv', 1273, 'S') #changed 1275 to 1273

In [113]:
print(s_protein_data)
#data looks good and is now all in float format

[0.0, 0.004, 0.008, 0.0, 0.132, 0.032, 0.0, 0.002, 0.019, 0.0, 0.0, 0.043, 0.063, 0.013999999999999999, 0.002, 0.005, 0.005, 0.28600000000000003, 0.175, 0.2, 0.013000000000000001, 0.02, 0.004, 0.0, 0.006, 0.19899999999999998, 0.064, 0.002, 0.015, 0.0, 0.0, 0.0, 0.009000000000000001, 0.0, 0.0, 0.004, 0.0, 0.0, 0.002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002, 0.0, 0.0, 0.026000000000000002, 0.0, 0.006, 0.079, 0.0, 0.011000000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002, 0.01, 0.002, 0.005, 0.102, 0.004, 0.628, 0.637, 0.013000000000000001, 0.013999999999999999, 0.006999999999999999, 0.005, 0.086, 0.073, 0.024, 0.0, 0.0, 0.254, 0.0, 0.002, 0.0, 0.002, 0.0, 0.0, 0.0, 0.004, 0.0, 0.002, 0.0, 0.0, 0.002, 0.018000000000000002, 0.165, 0.013000000000000001, 0.002, 0.052000000000000005, 0.0, 0.016, 0.0, 0.019, 0.0, 0.0, 0.002, 0.0, 0.0, 0.0, 0.0, 0.002, 0.004, 0.002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004, 0.002, 0.002, 0.002, 0.0, 0.0, 0.004, 0.0, 0.022000000000000002, 0.0, 0.002, 0.002, 0.005, 0

In [114]:
#removes fasta file header
def ref_seq_fasta_format(fasta_filename):
    with open(fasta_filename) as f:
        seq_lines = f.readlines()
    formatted_seq_lines = str(seq_lines[1:]) #was on the wrong indentation before.
    return formatted_seq_lines
    
    #formatted_seq_lines = removed_fasta_seq_lines.replace('/n', '')
    #this is just a piece of code i was thinking to use (doesnt work) to get rid of /n. maybe not even necessary.

In [115]:
ref_seq = ref_seq_fasta_format('sars_cov_2_spike_ref_seq.fasta')

In [116]:
print(ref_seq) #everything seems to check out.

['MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHV\n', 'SGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPF\n', 'LGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPI\n', 'NLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYN\n', 'ENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASV\n', 'YAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIAD\n', 'YNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYF\n', 'PLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFL\n', 'PFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLT\n', 'PTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLG\n', 'AENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGI\n', 'AVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDC\n', 'LGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIG\n', 'VTQNVLYENQ

In [117]:
#changed formatted_seq_lines to input_data which is the s_protein_data
def biostructmap_analysis(pdb_structure, gene_name, input_data, ref_sequence, output_filename):
    # Initialise structure object
    structure = biostructmap.Structure(pdb_structure, gene_name)

    data = {
        'A': 
            input_data,
        'B':
            input_data,
        'C':
            input_data,
    }
 
    #data = {chains: input_data}
    #commented this out for now until i get everything working (got confusing having it as a variable). 
    #just want it to work for this dataset at the moment.

    # Note: Use method='default' for mapping an ordered list of values (must contain values for each residue in the sequence.)
    results = structure.map(data, method='default', ref=ref_sequence, radius=15)

    results.write_data_to_pdb_b_factor(fileobj=output_filename)

In [ ]:
#seems like theres still something worng with the chain_id data. dont understand this: Output should be in the form {('A', 17): ('A', (' ', 24, ' ')), ...}
#changed seq_len to match nextstrain ref_seq. still got error that values dont match with residues.
#something wrong with matching residues to that in the .pdb. not sure?

In [118]:
biostructmap_analysis('6xm0.pdb', 'sars_cov_2_spike', s_protein_data, ref_seq, 'biostructmap_global_diversity.pdb')

/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 26695.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 26849.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 26975.
  PDBConstructionWarning,


AttributeError: ignored

In [ ]:
 with open('process_diversity_data.txt', 'w') as f:
        for entropy in list_of_values:
            f.write(f"{entropy:0.3f}\n")